In [1]:
from IPython.display import display, Markdown, Code
import tempfile
from pathlib import Path, PurePath
import os
import subprocess
import shlex
import hashlib
import zlib
import time
import shutil
from typing import Mapping, Tuple, Literal, Optional, List

In [2]:
base = tempfile.mkdtemp(prefix='git-mock-')

display(Code(f'cd {base}'))

cd /tmp/git-mock-nc4hzakb

In [3]:
def run_cmd(cmd):
    proc = subprocess.run(shlex.split(cmd), capture_output=True, encoding='utf-8')

    display(Code(f'>>> {shlex.join(proc.args)}\n{proc.stdout or proc.stderr}'))

In [4]:
run_cmd('git --version')

>>> git --version
git version 2.25.1

### git init

In [5]:
init_empty_dirs = [
    'objects/info',
    'refs/heads',
]

git_dir = PurePath(base).joinpath('.git')
os.mkdir(git_dir)

for d in init_empty_dirs:
    os.makedirs(git_dir.joinpath(d), exist_ok=True)

In [6]:
with open(git_dir.joinpath('config'), 'wt', encoding='utf-8') as f:
    f.write('''\
[core]
    repositoryformatversion = 0
    filemode = true
    bare = false
    logallrefupdates = true
''')

In [7]:
with open(git_dir.joinpath('HEAD'), 'wt', encoding='utf-8') as f:
    f.write('ref: refs/heads/master')

In [8]:
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
└── .git
    ├── config
    ├── HEAD
    ├── objects
    │   └── info
    └── refs
        └── heads

5 directories, 2 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)

### git add

In [9]:
py_v1_text = 'print("hello")\n'
py_file_name = 'first.txt'
with open(os.path.join(base, py_file_name), 'wt', encoding='utf-8') as f:
    f.write(py_v1_text)

In [10]:
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── objects
    │   └── info
    └── refs
        └── heads

5 directories, 3 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	first.txt

nothing added to commit but untracked files present (use "git add" to track)

In [11]:
def write_object(raw_content: bytes, sha1: str, git_dir: PurePath) -> None:
    compressed = zlib.compress(raw_content)
    object_dir = git_dir.joinpath('objects', sha1[:2])
    os.makedirs(object_dir, exist_ok=True)
    with open(object_dir.joinpath(sha1[2:]), 'wb') as f:
        f.write(compressed)

def write_blob_object(file_content: str) -> str:
    raw_content = f'blob {len(file_content)}\0{file_content}'.encode('utf-8')
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
        
    return sha1

In [12]:
with open(os.path.join(base, py_file_name), 'rt', encoding='utf-8') as f:
    file_content = f.read()
    
py_v1_blob_sha = write_blob_object(file_content)
print(py_v1_blob_sha)

11b15b1a4584b08fa423a57964bdbf018b0da0d5


In [13]:
run_cmd(f'git -C {base} cat-file -p {py_v1_blob_sha}')
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p 11b15b1a4584b08fa423a57964bdbf018b0da0d5
print("hello")

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── objects
    │   ├── 11
    │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
    │   └── info
    └── refs
        └── heads

6 directories, 4 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	first.txt

nothing added to commit but untracked files present (use "git add" to track)

In [14]:
"""
ref: https://git-scm.com/docs/index-format/2.25.0
"""
class IndexEntry:
    def __init__(self, path: str, blob_sha: str, base_path: str):
        self.path = path
        self.blob_sha = blob_sha
        self.base_path = base_path
    
    def to_bytes(self):
        stat = os.stat(self.path)
        
        b = int(stat.st_ctime).to_bytes(4, byteorder='big')
        b += int(stat.st_ctime_ns % 1e9).to_bytes(4, byteorder='big')
        b += int(stat.st_mtime).to_bytes(4, byteorder='big')
        b += int(stat.st_mtime_ns % 1e9).to_bytes(4, byteorder='big')
        
        b += int(stat.st_dev).to_bytes(4, byteorder='big')
        b += int(stat.st_ino).to_bytes(4, byteorder='big')
        b += int('100644', 8).to_bytes(4, byteorder='big')
        b += int(stat.st_uid).to_bytes(4, byteorder='big')
        b += int(stat.st_gid).to_bytes(4, byteorder='big')
        b += int(stat.st_size).to_bytes(4, byteorder='big')
        
        b += bytes.fromhex(self.blob_sha)
        
        assume_valid_flag = 0 << 3
        extended_flag = 0 << 2
        merge_stage_flag = 0
        name_length = len(os.path.basename(self.path)) if len(os.path.basename(self.path)) < 0xfff else 0xfff
        flags = (
            ((assume_valid_flag | extended_flag | merge_stage_flag) << 12) 
            | name_length
        ).to_bytes(2, byteorder='big')
        b += flags
        
        relative_path_name = os.path.relpath(self.path, self.base_path).encode('utf-8')
        b += relative_path_name
        
        padding_size = 8 - (len(b) % 8)
        b += (b'\0' * padding_size)
        
        return b

In [15]:
def write_index_file(entries: List[IndexEntry]) -> None:
    signature = b'DIRC'
    version = (2).to_bytes(4, byteorder='big')
    entries_number = len(entries).to_bytes(4, byteorder='big')
    
    entries = sorted(entries, key=lambda e: e.path)
    raw_content = signature + version + entries_number + b''.join([e.to_bytes() for e in entries])
    sha1 = hashlib.sha1(raw_content).hexdigest()
    raw_content += bytes.fromhex(sha1)
    
    with open(git_dir.joinpath('index'), 'wb') as f:
        f.write(raw_content)

In [16]:
py_v1_index_entry = IndexEntry(path=os.path.join(base, py_file_name), blob_sha=py_v1_blob_sha, base_path=base)
write_index_file([py_v1_index_entry])

In [17]:
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── index
    ├── objects
    │   ├── 11
    │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
    │   └── info
    └── refs
        └── heads

6 directories, 5 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   first.txt

### git commit

In [18]:
class TreeEntry:
    def __init__(self, object_type: str, name: str, sha: str):
        assert(object_type in ('tree', 'blob', 'commit', 'tag'))
        self.object_type = object_type
        self.name = name
        self.sha = sha
        self.mode = '100644' if object_type == 'blob' else '40000'

In [19]:
def write_tree_object(entries: List[TreeEntry]) -> str:
    sorted_entries = sorted(entries, key=lambda e: e.name)
    
    entries_content = b''.join([
        f'{e.mode} {e.name}\0'.encode('utf-8') + bytes.fromhex(e.sha) for e in sorted_entries
    ])
    raw_content = f'tree {len(entries_content)}\0'.encode('utf-8') + entries_content
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
    
    return sha1

In [20]:
py_tree_entry = TreeEntry(object_type='blob', name=py_file_name, sha=py_v1_blob_sha)
first_tree_sha = write_tree_object([py_tree_entry])
print(first_tree_sha)

86ee572420eee690fac4b69cd8ac700c837fd6e4


In [21]:
run_cmd(f'git -C {base} cat-file -p {first_tree_sha}')
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p 86ee572420eee690fac4b69cd8ac700c837fd6e4
100644 blob 11b15b1a4584b08fa423a57964bdbf018b0da0d5	first.txt

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── index
    ├── objects
    │   ├── 11
    │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
    │   ├── 86
    │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
    │   └── info
    └── refs
        └── heads

7 directories, 6 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   first.txt

In [22]:
my_name = 'Soros Liu'
my_email = 'soros.liu1029@gmail.com'

def write_commit_object(tree_sha: str, parent_commmit_sha: Optional[str], msg: str) -> str:
    commit = f'tree {tree_sha}\n' + \
        (f'parent {parent_commmit_sha}\n' if parent_commmit_sha else '') + \
        f'author {my_name} <{my_email}> {int(time.time())} +0800\n' + \
        f'committer {my_name} <{my_email}> {int(time.time())} +0800\n' + \
        '\n' + \
        msg + \
        '\n'
    
    commit_content = commit.encode('utf-8')
    raw_content = f'commit {len(commit_content)}\0'.encode('utf-8') + commit_content
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
    
    return sha1

In [23]:
first_commit_sha = write_commit_object(tree_sha=first_tree_sha, parent_commmit_sha=None, msg='first commit')
print(first_commit_sha)

2a69e57776966905ce1f953dd467739773a4ec0e


In [24]:
run_cmd(f'git -C {base} cat-file -p {first_commit_sha}')
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')
run_cmd(f'git -C {base} log')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p 2a69e57776966905ce1f953dd467739773a4ec0e
tree 86ee572420eee690fac4b69cd8ac700c837fd6e4
author Soros Liu <soros.liu1029@gmail.com> 1634724484 +0800
committer Soros Liu <soros.liu1029@gmail.com> 1634724484 +0800

first commit

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── index
    ├── objects
    │   ├── 11
    │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
    │   ├── 2a
    │   │   └── 69e57776966905ce1f953dd467739773a4ec0e
    │   ├── 86
    │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
    │   └── info
    └── refs
        └── heads

8 directories, 7 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   first.txt

>>> git -C /tmp/git-mock-nc4hzakb log
fatal: your current branch 'master' does not have any commits yet

### bookmark

In [25]:
with open(git_dir.joinpath('refs', 'heads', 'master'), 'wt', encoding='utf-8') as f:
    f.write(first_commit_sha)

In [26]:
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')
run_cmd(f'git -C {base} log')
run_cmd(f'git -C {base} fsck --verbose')

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
└── .git
    ├── config
    ├── HEAD
    ├── index
    ├── objects
    │   ├── 11
    │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
    │   ├── 2a
    │   │   └── 69e57776966905ce1f953dd467739773a4ec0e
    │   ├── 86
    │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
    │   └── info
    └── refs
        └── heads
            └── master

8 directories, 8 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
nothing to commit, working tree clean

>>> git -C /tmp/git-mock-nc4hzakb log
commit 2a69e57776966905ce1f953dd467739773a4ec0e
Author: Soros Liu <soros.liu1029@gmail.com>
Date:   Wed Oct 20 18:08:04 2021 +0800

    first commit

>>> git -C /tmp/git-mock-nc4hzakb fsck --verbose
Checking object directory
Checking blob 11b15b1a4584b08fa423a57964bdbf018b0da0d5
Checking commit 2a69e57776966905ce1f953dd467739773a4ec0e
Checking tree 86ee572420eee690fac4b69cd8ac700c837fd6e4
Checking HEAD link
Checking connectivity (32 objects)
Checking 86ee572420eee690fac4b69cd8ac700c837fd6e4
Checking 2a69e57776966905ce1f953dd467739773a4ec0e
Checking 11b15b1a4584b08fa423a57964bdbf018b0da0d5

### add more files

In [27]:
md_text = '## Explore Git\n'
md_file_name = 'README.md'
with open(os.path.join(base, md_file_name), 'wt', encoding='utf-8') as f:
    f.write(md_text)

In [28]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md

nothing added to commit but untracked files present (use "git add" to track)

In [29]:
with open(os.path.join(base, md_file_name), 'rt', encoding='utf-8') as f:
    file_content = f.read()
    
md_blob_sha = write_blob_object(file_content)
print(md_blob_sha)

29ed52ec5fa9b6631a198103754707735d637187


In [30]:
run_cmd(f'git -C {base} cat-file -p {md_blob_sha}')
run_cmd(f'git -C {base} status')
run_cmd(f'tree {base} -a')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p 29ed52ec5fa9b6631a198103754707735d637187
## Explore Git

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	README.md

nothing added to commit but untracked files present (use "git add" to track)

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
├── .git
│   ├── config
│   ├── HEAD
│   ├── index
│   ├── objects
│   │   ├── 11
│   │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
│   │   ├── 29
│   │   │   └── ed52ec5fa9b6631a198103754707735d637187
│   │   ├── 2a
│   │   │   └── 69e57776966905ce1f953dd467739773a4ec0e
│   │   ├── 86
│   │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
│   │   └── info
│   └── refs
│       └── heads
│           └── master
└── README.md

9 directories, 10 files

In [31]:
md_index_entry = IndexEntry(path=os.path.join(base, md_file_name), blob_sha=md_blob_sha, base_path=base)
write_index_file([py_v1_index_entry, md_index_entry])

In [32]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   README.md

In [33]:
md_tree_entry = TreeEntry(object_type='blob', name=md_file_name, sha=md_blob_sha)
second_tree_sha = write_tree_object([py_tree_entry, md_tree_entry])
print(second_tree_sha)

da46d73aa5a4c8a9892f7e35583db437d374a24c


In [34]:
run_cmd(f'git -C {base} cat-file -p {second_tree_sha}')
run_cmd(f'tree {base} -a')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p da46d73aa5a4c8a9892f7e35583db437d374a24c
100644 blob 29ed52ec5fa9b6631a198103754707735d637187	README.md
100644 blob 11b15b1a4584b08fa423a57964bdbf018b0da0d5	first.txt

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
├── .git
│   ├── config
│   ├── HEAD
│   ├── index
│   ├── objects
│   │   ├── 11
│   │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
│   │   ├── 29
│   │   │   └── ed52ec5fa9b6631a198103754707735d637187
│   │   ├── 2a
│   │   │   └── 69e57776966905ce1f953dd467739773a4ec0e
│   │   ├── 86
│   │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
│   │   ├── da
│   │   │   └── 46d73aa5a4c8a9892f7e35583db437d374a24c
│   │   └── info
│   └── refs
│       └── heads
│           └── master
└── README.md

10 directories, 11 files

In [35]:
second_commit_sha = write_commit_object(tree_sha=second_tree_sha, parent_commmit_sha=first_commit_sha, msg='second commit')
print(second_commit_sha)

cc3b09e280d24808d82a00016be53687cf22e220


In [36]:
run_cmd(f'git -C {base} cat-file -p {second_commit_sha}')
run_cmd(f'tree {base} -a')
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb cat-file -p cc3b09e280d24808d82a00016be53687cf22e220
tree da46d73aa5a4c8a9892f7e35583db437d374a24c
parent 2a69e57776966905ce1f953dd467739773a4ec0e
author Soros Liu <soros.liu1029@gmail.com> 1634724484 +0800
committer Soros Liu <soros.liu1029@gmail.com> 1634724484 +0800

second commit

>>> tree /tmp/git-mock-nc4hzakb -a
/tmp/git-mock-nc4hzakb
├── first.txt
├── .git
│   ├── config
│   ├── HEAD
│   ├── index
│   ├── objects
│   │   ├── 11
│   │   │   └── b15b1a4584b08fa423a57964bdbf018b0da0d5
│   │   ├── 29
│   │   │   └── ed52ec5fa9b6631a198103754707735d637187
│   │   ├── 2a
│   │   │   └── 69e57776966905ce1f953dd467739773a4ec0e
│   │   ├── 86
│   │   │   └── ee572420eee690fac4b69cd8ac700c837fd6e4
│   │   ├── cc
│   │   │   └── 3b09e280d24808d82a00016be53687cf22e220
│   │   ├── da
│   │   │   └── 46d73aa5a4c8a9892f7e35583db437d374a24c
│   │   └── info
│   └── refs
│       └── heads
│           └── master
└── README.md

11 directories, 12 files

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   README.md

In [37]:
with open(git_dir.joinpath('refs', 'heads', 'master'), 'wt', encoding='utf-8') as f:
    f.write(second_commit_sha)

In [38]:
run_cmd(f'git -C {base} status')
run_cmd(f'git -C {base} log')
run_cmd(f'git -C {base} fsck --verbose')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch master
nothing to commit, working tree clean

>>> git -C /tmp/git-mock-nc4hzakb log
commit cc3b09e280d24808d82a00016be53687cf22e220
Author: Soros Liu <soros.liu1029@gmail.com>
Date:   Wed Oct 20 18:08:04 2021 +0800

    second commit

commit 2a69e57776966905ce1f953dd467739773a4ec0e
Author: Soros Liu <soros.liu1029@gmail.com>
Date:   Wed Oct 20 18:08:04 2021 +0800

    first commit

>>> git -C /tmp/git-mock-nc4hzakb fsck --verbose
Checking object directory
Checking blob 11b15b1a4584b08fa423a57964bdbf018b0da0d5
Checking blob 29ed52ec5fa9b6631a198103754707735d637187
Checking commit 2a69e57776966905ce1f953dd467739773a4ec0e
Checking tree 86ee572420eee690fac4b69cd8ac700c837fd6e4
Checking commit cc3b09e280d24808d82a00016be53687cf22e220
Checking tree da46d73aa5a4c8a9892f7e35583db437d374a24c
Checking HEAD link
Checking connectivity (32 objects)
Checking 86ee572420eee690fac4b69cd8ac700c837fd6e4
Checking 29ed52ec5fa9b6631a198103754707735d637187
Checking 2a69e57776966905ce1f953dd467739773a4ec0e
Checking cc3b09e280d24808d82a00016be53687cf22e220
Checking 11b15b1a4584b08fa423a57964bdbf018b0da0d5
Checking da46d73aa5a4c8a9892f7e35583db437d374a24c

### git checkout -b

In [39]:
run_cmd(f'cat {base}/.git/HEAD')
run_cmd(f'git -C {base} branch --show-current')
run_cmd(f'git -C {base} rev-parse HEAD')

>>> cat /tmp/git-mock-nc4hzakb/.git/HEAD
ref: refs/heads/master

>>> git -C /tmp/git-mock-nc4hzakb branch --show-current
master

>>> git -C /tmp/git-mock-nc4hzakb rev-parse HEAD
cc3b09e280d24808d82a00016be53687cf22e220

In [40]:
shutil.copy(git_dir.joinpath('refs', 'heads', 'master'), git_dir.joinpath('refs', 'heads', 'new-idea'))

with open(git_dir.joinpath('HEAD'), 'wt', encoding='utf-8') as f:
    f.write('ref: refs/heads/new-idea')

In [41]:
run_cmd(f'git -C {base} branch --show-current')
run_cmd(f'git -C {base} rev-parse HEAD')
run_cmd(f'git -C {base} log --oneline --decorate --graph')

>>> git -C /tmp/git-mock-nc4hzakb branch --show-current
new-idea

>>> git -C /tmp/git-mock-nc4hzakb rev-parse HEAD
cc3b09e280d24808d82a00016be53687cf22e220

>>> git -C /tmp/git-mock-nc4hzakb log --oneline --decorate --graph
* cc3b09e (HEAD -> new-idea, master) second commit
* 2a69e57 first commit

In [42]:
py_v2_text = 'print("hello world")\n'
with open(os.path.join(base, py_file_name), 'wt', encoding='utf-8') as f:
    f.write(py_v2_text)

In [43]:
run_cmd(f'git -C {base} status')
run_cmd(f'git -C {base} diff')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch new-idea
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   first.txt

no changes added to commit (use "git add" and/or "git commit -a")

>>> git -C /tmp/git-mock-nc4hzakb diff
diff --git a/first.txt b/first.txt
index 11b15b1..8cde782 100644
--- a/first.txt
+++ b/first.txt
@@ -1 +1 @@
-print("hello")
+print("hello world")

In [44]:
with open(os.path.join(base, py_file_name), 'rt', encoding='utf-8') as f:
    file_content = f.read()
    
py_v2_blob_sha = write_blob_object(file_content)
print(py_v2_blob_sha)

8cde7829c178ede96040e03f17c416d15bdacd01


In [45]:
py_v2_index_entry = IndexEntry(path=os.path.join(base, py_file_name), blob_sha=py_v2_blob_sha, base_path=base)
write_index_file([py_v2_index_entry, md_index_entry])

In [46]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch new-idea
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   first.txt

In [47]:
py_tree_entry = TreeEntry(object_type='blob', name=py_file_name, sha=py_v2_blob_sha)
third_tree_sha = write_tree_object([py_tree_entry, md_tree_entry])
print(third_tree_sha)

third_commit_sha = write_commit_object(tree_sha=third_tree_sha, parent_commmit_sha=second_commit_sha, msg='third commit')
print(third_commit_sha)

with open(git_dir.joinpath('refs', 'heads', 'new-idea'), 'wt', encoding='utf-8') as f:
    f.write(third_commit_sha)

cde782d8509e0b5d4c45922724f9cb1dbd77b44c
58499a7089328763174c8a4cde14034880ce318b


In [48]:
run_cmd(f'git -C {base} status')
run_cmd(f'git -C {base} log --oneline --decorate --graph')

>>> git -C /tmp/git-mock-nc4hzakb status
On branch new-idea
nothing to commit, working tree clean

>>> git -C /tmp/git-mock-nc4hzakb log --oneline --decorate --graph
* 58499a7 (HEAD -> new-idea) third commit
* cc3b09e (master) second commit
* 2a69e57 first commit